In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GEMINI_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# GEPA Prompt Optimization with Images

This notebook demonstrates how to use GEPA with DSPy adapter to optimize prompts for image-based tasks.

## Setup
- **Task LLM**: Gemini 2.0 Flash Lite
- **Reflection LLM**: GPT-5 (high reasoning)
- **Dataset**: Images + Prompts → Expected JSON output

---

## 🎯 Quick Start: Single-Worksheet Optimization

**This notebook optimizes ONE worksheet type at a time.**

### Current Configuration:
- **Worksheet Index**: Set in cell 16 (`WORKSHEET_INDEX = 1`)
- **Optimization Mode**: Individual (no generalization)
- **Iterations**: 100 (adjustable in cell 24)

### To Optimize Different Worksheets:
1. **Worksheet 1**: Set `WORKSHEET_INDEX = 1` → Run All
2. **Worksheet 2**: Set `WORKSHEET_INDEX = 2` → Run All  
3. **Worksheet 3**: Set `WORKSHEET_INDEX = 3` → Run All

Each worksheet gets its own optimized program file.

---

In [ ]:
# Install required packages
#!pip install gepa dspy-ai pillow openai google-generativeai python-dotenv

In [ ]:
import json
import os
from pathlib import Path
from typing import List, Dict, Any, Tuple
from PIL import Image
import base64
from io import BytesIO
import dspy
from gepa import optimize
from gepa.adapters.dspy_full_program_adapter.full_program_adapter import DspyAdapter

## Dataset Structure

The Dataset folder should have the following structure:
```
Dataset/
├── prompts/
│   ├── 1.txt
│   ├── 2.txt
│   └── ...
├── outputs/
│   ├── 1.json
│   ├── 2.json
│   └── ...
└── images/
    ├── 1-a.png (or .jpg, .jpeg)
    ├── 1-b.png (or .jpg, .jpeg)
    ├── 2-a.jpg
    ├── 2-b.jpg
    └── ...
```

**Supported image formats**: PNG, JPG, JPEG (case-insensitive)

In [ ]:
def encode_image_to_base64(image_path: str) -> str:
    """Convert image to base64 string for API consumption. Supports PNG, JPG, JPEG."""
    with Image.open(image_path) as img:
        # Convert RGBA to RGB if necessary (for JPG compatibility)
        if img.mode == 'RGBA':
            # Create white background
            background = Image.new('RGB', img.size, (255, 255, 255))
            background.paste(img, mask=img.split()[3])  # Use alpha channel as mask
            img = background
        
        buffered = BytesIO()
        # Detect format from file extension
        img_format = Path(image_path).suffix.lower()
        if img_format in ['.jpg', '.jpeg']:
            img.save(buffered, format="JPEG", quality=95)
        else:
            img.save(buffered, format="PNG")
        return base64.b64encode(buffered.getvalue()).decode('utf-8')


def load_dataset_item(idx: int, dataset_path: str = "Dataset") -> Dict[str, Any]:
    """
    Load a single dataset item with prompt, images, and expected output.
    
    Args:
        idx: Item index (1-based)
        dataset_path: Path to Dataset folder
        
    Returns:
        Dictionary containing prompt, images, and expected JSON output
    """
    dataset_path = Path(dataset_path)
    
    # Load prompt
    prompt_file = dataset_path / "prompts" / f"{idx}.txt"
    with open(prompt_file, 'r', encoding='utf-8') as f:
        prompt = f.read().strip()
    
    # Load expected output
    output_file = dataset_path / "outputs" / f"{idx}.json"
    with open(output_file, 'r', encoding='utf-8') as f:
        expected_output = json.load(f)
    
    # Load images (find all images for this index - supports PNG, JPG, JPEG)
    images_dir = dataset_path / "images"
    image_files = []
    for ext in ['*.png', '*.jpg', '*.jpeg', '*.PNG', '*.JPG', '*.JPEG']:
        image_files.extend(images_dir.glob(f"{idx}-{ext}"))
    
    # Remove duplicates (case-insensitive filesystem can cause duplicates)
    seen = set()
    unique_files = []
    for f in sorted(image_files):
        if f.name.lower() not in seen:
            seen.add(f.name.lower())
            unique_files.append(f)
    image_files = unique_files
    
    images = []
    for img_path in image_files:
        images.append({
            "path": str(img_path),
            "base64": encode_image_to_base64(str(img_path))
        })
    
    return {
        "index": idx,
        "prompt": prompt,
        "images": images,
        "expected_output": expected_output
    }


def load_all_dataset(dataset_path: str = "Dataset", max_items: int = None) -> List[Dict[str, Any]]:
    """
    Load all dataset items.
    
    Args:
        dataset_path: Path to Dataset folder
        max_items: Maximum number of items to load (None for all)
        
    Returns:
        List of dataset items
    """
    dataset_path = Path(dataset_path)
    prompts_dir = dataset_path / "prompts"
    
    # Get all prompt files
    prompt_files = sorted(prompts_dir.glob("*.txt"))
    indices = [int(f.stem) for f in prompt_files]
    
    if max_items:
        indices = indices[:max_items]
    
    dataset = []
    for idx in indices:
        try:
            item = load_dataset_item(idx, str(dataset_path))
            dataset.append(item)
        except Exception as e:
            print(f"Error loading item {idx}: {e}")
    
    return dataset

## Validate Dataset Structure

Run this to check if your Dataset folder is correctly set up.

In [ ]:
def validate_dataset(dataset_path: str = "Dataset") -> Dict[str, Any]:
    """
    Validate the Dataset folder structure and contents.
    
    Checks:
    - Folder structure exists
    - Prompts, outputs, and images are properly matched
    - File formats are correct
    - JSON files are valid
    - Images can be loaded
    
    Returns:
        Dictionary with validation results and issues found
    """
    dataset_path = Path(dataset_path)
    results = {
        "valid": True,
        "issues": [],
        "warnings": [],
        "summary": {},
        "worksheets": {}
    }
    
    print("="*80)
    print("DATASET VALIDATION")
    print("="*80)
    
    # Check if Dataset folder exists
    if not dataset_path.exists():
        results["valid"] = False
        results["issues"].append(f"❌ Dataset folder not found: {dataset_path}")
        print(f"❌ Dataset folder not found: {dataset_path}")
        return results
    
    print(f"✓ Dataset folder found: {dataset_path}")
    
    # Check required subfolders
    prompts_dir = dataset_path / "prompts"
    outputs_dir = dataset_path / "outputs"
    images_dir = dataset_path / "images"
    
    for folder_name, folder_path in [("prompts", prompts_dir), ("outputs", outputs_dir), ("images", images_dir)]:
        if not folder_path.exists():
            results["valid"] = False
            results["issues"].append(f"❌ Missing '{folder_name}' folder")
            print(f"❌ Missing '{folder_name}' folder")
        else:
            print(f"✓ Found '{folder_name}' folder")
    
    if not results["valid"]:
        return results
    
    print("\n" + "-"*80)
    print("CHECKING WORKSHEETS")
    print("-"*80)
    
    # Get all prompt files
    prompt_files = sorted(prompts_dir.glob("*.txt"))
    
    if not prompt_files:
        results["valid"] = False
        results["issues"].append("❌ No prompt files (.txt) found in prompts folder")
        print("❌ No prompt files (.txt) found in prompts folder")
        return results
    
    print(f"\n✓ Found {len(prompt_files)} prompt file(s)")
    
    # Check each worksheet
    for prompt_file in prompt_files:
        worksheet_idx = prompt_file.stem
        worksheet_info = {
            "prompt_file": str(prompt_file),
            "issues": [],
            "warnings": [],
            "valid": True
        }
        
        print(f"\n{'='*80}")
        print(f"Worksheet: {worksheet_idx}")
        print(f"{'='*80}")
        
        # Check prompt file
        try:
            with open(prompt_file, 'r', encoding='utf-8') as f:
                prompt_content = f.read().strip()
            
            if not prompt_content:
                worksheet_info["issues"].append("⚠️  Prompt file is empty")
                worksheet_info["valid"] = False
                print(f"⚠️  Prompt file is empty")
            else:
                print(f"✓ Prompt file: {len(prompt_content)} characters")
                worksheet_info["prompt_length"] = len(prompt_content)
        except Exception as e:
            worksheet_info["issues"].append(f"❌ Error reading prompt file: {e}")
            worksheet_info["valid"] = False
            print(f"❌ Error reading prompt file: {e}")
        
        # Check output JSON file
        output_file = outputs_dir / f"{worksheet_idx}.json"
        if not output_file.exists():
            worksheet_info["issues"].append("❌ Missing corresponding .json file in outputs folder")
            worksheet_info["valid"] = False
            print(f"❌ Missing corresponding .json file in outputs folder")
        else:
            try:
                with open(output_file, 'r', encoding='utf-8') as f:
                    output_data = json.load(f)
                
                if not output_data:
                    worksheet_info["warnings"].append("⚠️  Output JSON is empty")
                    print(f"⚠️  Output JSON is empty")
                else:
                    # Handle both list and dict formats
                    if isinstance(output_data, list):
                        print(f"✓ Output JSON: List with {len(output_data)} items")
                        if output_data and isinstance(output_data[0], dict):
                            print(f"  First item keys: {list(output_data[0].keys())}")
                            worksheet_info["output_type"] = "list"
                            worksheet_info["output_count"] = len(output_data)
                    elif isinstance(output_data, dict):
                        print(f"✓ Output JSON: Dict with {len(output_data)} key(s) - {list(output_data.keys())}")
                        worksheet_info["output_keys"] = list(output_data.keys())
                        worksheet_info["output_type"] = "dict"
            except json.JSONDecodeError as e:
                worksheet_info["issues"].append(f"❌ Invalid JSON format: {e}")
                worksheet_info["valid"] = False
                print(f"❌ Invalid JSON format: {e}")
            except Exception as e:
                worksheet_info["issues"].append(f"❌ Error reading output file: {e}")
                worksheet_info["valid"] = False
                print(f"❌ Error reading output file: {e}")
        
        # Check image files
        image_files = []
        for ext in ['*.png', '*.jpg', '*.jpeg', '*.PNG', '*.JPG', '*.JPEG']:
            image_files.extend(images_dir.glob(f"{worksheet_idx}-{ext}"))
        
        # Remove duplicates (case-insensitive filesystem can cause duplicates)
        seen = set()
        unique_files = []
        for f in sorted(image_files):
            if f.name.lower() not in seen:
                seen.add(f.name.lower())
                unique_files.append(f)
        image_files = unique_files
        
        if not image_files:
            worksheet_info["issues"].append("❌ No image files found (expected format: {worksheet_idx}-a.png, {worksheet_idx}-b.jpg, etc.)")
            worksheet_info["valid"] = False
            print(f"❌ No image files found (expected format: {worksheet_idx}-a.png, {worksheet_idx}-b.jpg, etc.)")
        else:
            print(f"✓ Found {len(image_files)} image(s):")
            worksheet_info["images"] = []
            
            for img_file in image_files:
                try:
                    # Try to open and validate image
                    with Image.open(img_file) as img:
                        img_info = {
                            "file": img_file.name,
                            "format": img.format,
                            "size": f"{img.width}x{img.height}",
                            "mode": img.mode
                        }
                        worksheet_info["images"].append(img_info)
                        print(f"  ✓ {img_file.name} - {img.format} {img.width}x{img.height} ({img.mode})")
                except Exception as e:
                    worksheet_info["issues"].append(f"❌ Error loading image {img_file.name}: {e}")
                    worksheet_info["valid"] = False
                    print(f"  ❌ Error loading image {img_file.name}: {e}")
        
        # Store worksheet info
        results["worksheets"][worksheet_idx] = worksheet_info
        
        if not worksheet_info["valid"]:
            results["valid"] = False
        
        # Summary for this worksheet
        if worksheet_info["valid"]:
            print(f"\n✅ Worksheet {worksheet_idx} is valid!")
        else:
            print(f"\n❌ Worksheet {worksheet_idx} has issues!")
            print(f"\n❌ Worksheet {worksheet_idx} has issues!")
    # Overall summary
    print("\n" + "="*80)
    print("VALIDATION SUMMARY")
    print("="*80)
    
    valid_worksheets = sum(1 for w in results["worksheets"].values() if w["valid"])
    total_worksheets = len(results["worksheets"])
    
    results["summary"] = {
        "total_worksheets": total_worksheets,
        "valid_worksheets": valid_worksheets,
        "invalid_worksheets": total_worksheets - valid_worksheets
    }
    
    print(f"Total Worksheets: {total_worksheets}")
    print(f"Valid: {valid_worksheets}")
    print(f"Invalid: {total_worksheets - valid_worksheets}")
    
    if results["valid"]:
        print(f"\n✅ All worksheets are valid! Ready for optimization.")
    else:
        print(f"\n❌ Some worksheets have issues. Please fix them before optimization.")
        print(f"\nIssues found:")
        for ws_idx, ws_info in results["worksheets"].items():
            if ws_info["issues"]:
                print(f"\n  Worksheet {ws_idx}:")
                for issue in ws_info["issues"]:
                    print(f"    {issue}")
    
    print("="*80)
    
    return results








# #     print(validation_results["worksheets"]["1"])# # if "1" in validation_results["worksheets"]:# # To check a specific worksheet:# validation_results = validate_dataset("Dataset")# Run validation    return results


# Run validation
validation_results = validate_dataset("Dataset")
# 
# # To check a specific worksheet:
# # if "1" in validation_results["worksheets"]:
# #     print(validation_results["worksheets"]["1"])

## ⚡ Quick Validation Check

Run this cell to validate your Dataset folder before optimization.

In [ ]:
# Validate Dataset structure
validation_results = validate_dataset("Dataset")

# Store for later reference
dataset_is_valid = validation_results["valid"]

if dataset_is_valid:
    print("\n🎉 Your dataset is ready for optimization!")
else:
    print("\n⚠️  Please fix the issues above before running optimization.")

## DSPy Program for Image Processing

This defines a DSPy program that takes images and a prompt, processes them with vision-capable LLM, and returns JSON output.

In [ ]:
# Initial seed program for GEPA optimization
# This uses a custom vision module that actually sends images to the vision API
seed_program = """
import dspy
import json
from typing import List, Dict, Any
from pathlib import Path
import os
from openai import OpenAI
from pydantic import BaseModel, Field

class ExtractedQuestion(BaseModel):
    question_number: int = Field(description="The unique identifier for the question")
    question: str = Field(description="The entire text of the question without the student's answer")
    student_answer: str = Field(description="The student's answer exactly as written")

class VisionImageAnalysis(dspy.Signature):
    '''Extract questions and student answers from worksheet images using vision capabilities.
    
    Analyze the provided worksheet images and extract EVERY question with:
    - question_number: Sequential number (1, 2, 3, ...)
    - question: The COMPLETE question text (e.g., "Q1. __ , 31" not "What comes before 31?")
    - student_answer: EXACTLY what the student wrote in pencil
    
    Return ONLY a valid JSON array. Do NOT add any explanatory text.
    '''
    
    prompt: str = dspy.InputField(desc="Instructions for analyzing the worksheet images")
    images_info: str = dspy.InputField(desc="Metadata about the images being analyzed")
    json_output: str = dspy.OutputField(desc="JSON array containing extracted questions and answers")

class ImageAnalysisProgram(dspy.Module):
    def __init__(self):
        super().__init__()
        self.signature = VisionImageAnalysis
    
    def forward(self, prompt: str, images_info: str, images: List[Dict] = None):
        '''Execute vision analysis with actual image data'''
        if not images:
            return dspy.Prediction(json_output='[]')
        
        # Call OpenAI vision API with structured output
        client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        
        # Build message with images
        content = [{"type": "text", "text": prompt}]
        
        for img in images:
            img_path = Path(img.get('path', ''))
            ext = img_path.suffix.lower()
            mime_type = "image/jpeg" if ext in ['.jpg', '.jpeg'] else "image/png"
            
            content.append({
                "type": "image_url",
                "image_url": {"url": f"data:{mime_type};base64,{img['base64']}"}
            })
        
        # Call with response_format for JSON
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": content}],
            temperature=0.1,  # Lower for more consistent JSON
            max_tokens=4096,
            response_format={"type": "json_object"}  # Force JSON output
        )
        
        output = response.choices[0].message.content
        
        # Try to extract JSON array if wrapped
        output = output.strip()
        
        # If output is wrapped in a JSON object, extract the array
        try:
            parsed = json.loads(output)
            if isinstance(parsed, dict) and 'questions' in parsed:
                output = json.dumps(parsed['questions'])
            elif isinstance(parsed, dict) and 'results' in parsed:
                output = json.dumps(parsed['results'])
            elif isinstance(parsed, dict) and 'data' in parsed:
                output = json.dumps(parsed['data'])
        except:
            pass
        
        return dspy.Prediction(json_output=output)

program = ImageAnalysisProgram()
"""

## Execution Function with Vision API

This function executes the prompt with images using vision-capable models.

In [ ]:
def execute_vision_prompt(prompt: str, images: List[Dict], system_prompt: str = "") -> str:
    """
    Execute a prompt with images using OpenAI's vision API (gpt-4o).
    
    Args:
        prompt: The user prompt
        images: List of image dictionaries with 'base64' field
        system_prompt: Optional system prompt for context
        
    Returns:
        JSON string output from the model
    """
    from openai import OpenAI
    
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    # Build message content with images
    content = []
    
    if system_prompt:
        content.append({"type": "text", "text": system_prompt})
    
    content.append({"type": "text", "text": prompt})
    
    # Add all images with proper MIME type detection
    for img in images:
        # Detect image format from file path
        img_path = Path(img.get('path', ''))
        ext = img_path.suffix.lower()
        
        if ext in ['.jpg', '.jpeg']:
            mime_type = "image/jpeg"
        else:
            mime_type = "image/png"
        
        content.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:{mime_type};base64,{img['base64']}"
            }
        })
    
    # Call vision API
    response = client.chat.completions.create(
        model="gpt-4o",  # This is the flash-lite equivalent
        messages=[
            {"role": "user", "content": content}
        ],
        temperature=0.7,
        max_tokens=4096
    )
    
    return response.choices[0].message.content


def execute_with_gemini(prompt: str, images: List[Dict], system_prompt: str = "") -> str:
    """
    Alternative: Execute with Google Gemini (gpt-5 high equivalent for reasoning).
    
    Args:
        prompt: The user prompt
        images: List of image dictionaries with 'path' field
        system_prompt: Optional system prompt for context
        
    Returns:
        JSON string output from the model
    """
    import google.generativeai as genai
    
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
    
    # Use Gemini 2.0 Flash (high reasoning capability)
    model = genai.GenerativeModel('gemini-2.0-flash-exp')
    
    # Build content with images
    content_parts = []
    
    if system_prompt:
        content_parts.append(system_prompt + "\n\n")
    
    content_parts.append(prompt)
    
    # Add images
    for img in images:
        pil_img = Image.open(img['path'])
        content_parts.append(pil_img)
    
    response = model.generate_content(content_parts)
    return response.text

## Metric Function for Evaluation

This function compares the model output with expected JSON output.

In [ ]:
def compare_json_outputs(predicted: str, expected) -> float:
    """
    Compare predicted JSON output with expected output.
    Supports both dict and list expected outputs.
    
    Args:
        predicted: Predicted JSON string from model
        expected: Expected JSON (dict or list)
        
    Returns:
        Score between 0 and 1 (1 = perfect match)
    """
    try:
        # Parse predicted JSON
        predicted_json = json.loads(predicted)
        
        # Exact match check
        if predicted_json == expected:
            return 1.0
        
        # Handle list outputs (array of objects)
        if isinstance(expected, list) and isinstance(predicted_json, list):
            if len(expected) == 0:
                return 0.0
            
            # Compare list elements
            score = 0.0
            max_len = max(len(expected), len(predicted_json))
            
            for i in range(min(len(expected), len(predicted_json))):
                expected_item = expected[i]
                predicted_item = predicted_json[i]
                
                if expected_item == predicted_item:
                    score += 1.0 / max_len
                elif isinstance(expected_item, dict) and isinstance(predicted_item, dict):
                    # Partial credit for matching keys in dict items
                    item_score = 0.0
                    for key in expected_item:
                        if key in predicted_item:
                            if predicted_item[key] == expected_item[key]:
                                item_score += 1.0 / len(expected_item)
                            else:
                                item_score += 0.3 / len(expected_item)
                    score += item_score / max_len
            
            return score
        
        # Handle dict outputs
        if isinstance(expected, dict) and isinstance(predicted_json, dict):
            score = 0.0
            total_keys = len(expected)
            
            if total_keys == 0:
                return 0.0
            
            for key in expected:
                if key in predicted_json:
                    # Check if values match (for simple comparison)
                    if predicted_json[key] == expected[key]:
                        score += 1.0 / total_keys
                    else:
                        # Partial credit for having the key
                        score += 0.5 / total_keys
            
            return score
        
        # Type mismatch
        return 0.0
        
    except json.JSONDecodeError:
        # Invalid JSON - try string matching for recovery
        score = 0.0
        if isinstance(expected, dict):
            for key, value in expected.items():
                if str(key) in predicted and str(value) in predicted:
                    score += 0.3 / len(expected)
        elif isinstance(expected, list):
            # Check if any expected items appear as strings
            for item in expected[:5]:  # Check first 5 items
                if str(item) in predicted:
                    score += 0.1 / min(len(expected), 5)
        return score
    except Exception as e:
        print(f"Error comparing outputs: {e}")
        return 0.0


def metric_fn(example, pred, trace=None) -> float:
    """
    Metric function for DSPy adapter.
    
    Args:
        example: DSPy Example with expected_output field
        pred: Prediction with json_output field
        trace: Optional trace information
        
    Returns:
        Score between 0 and 1
    """
    try:
        predicted_output = pred.json_output if hasattr(pred, 'json_output') else str(pred)
        expected_output = example.expected_output if hasattr(example, 'expected_output') else {}
        
        return compare_json_outputs(predicted_output, expected_output)
    except Exception as e:
        print(f"Metric evaluation error: {e}")
        return 0.0

## Prepare Dataset for DSPy

Convert dataset items to DSPy Examples with image context.

In [ ]:
def prepare_dspy_dataset(dataset: List[Dict[str, Any]]) -> List[dspy.Example]:
    """
    Convert dataset items to DSPy Examples.
    
    Args:
        dataset: List of dataset items from load_all_dataset
        
    Returns:
        List of DSPy Examples
    """
    dspy_examples = []
    
    for item in dataset:
        # Create image context description
        images_info = f"Number of images: {len(item['images'])}"
        for i, img in enumerate(item['images'], 1):
            images_info += f"\n- Image {i}: {img['path']}"
        
        # Create DSPy Example - IMPORTANT: Pass raw images so vision model can access them
        example = dspy.Example(
            prompt=item['prompt'],
            images_info=images_info,
            images=item['images'],  # Pass actual image data with base64
            expected_output=item['expected_output']
        ).with_inputs('prompt', 'images_info', 'images')  # Include 'images' as input
        
        dspy_examples.append(example)
    
    return dspy_examples

## Load and Split Dataset

Load the dataset and split into train/validation sets.

In [ ]:
# ============================================================
# CONFIGURATION: Single Worksheet Optimization Mode
# ============================================================
# Set the worksheet index you want to optimize (e.g., 1, 2, 3, etc.)
WORKSHEET_INDEX = 1  # Change this to optimize different worksheet types

# Load single worksheet for optimization
print(f"Loading worksheet {WORKSHEET_INDEX} for individual optimization...")
single_item = load_dataset_item(WORKSHEET_INDEX, "Dataset")
print(f"✓ Loaded worksheet {WORKSHEET_INDEX}")
print(f"  - Prompt length: {len(single_item['prompt'])} chars")
print(f"  - Number of images: {len(single_item['images'])}")

# Handle both list and dict expected outputs
expected_output = single_item['expected_output']
if isinstance(expected_output, list):
    print(f"  - Expected output: List with {len(expected_output)} items")
    if expected_output:
        print(f"  - First item keys: {list(expected_output[0].keys())}")
elif isinstance(expected_output, dict):
    print(f"  - Expected output keys: {list(expected_output.keys())}")
else:
    print(f"  - Expected output type: {type(expected_output)}")

# Convert to DSPy format
dspy_dataset = prepare_dspy_dataset([single_item])

# For single worksheet optimization: use the same example for both train and val
# This allows GEPA to iterate and refine specifically for this worksheet type
train_set = dspy_dataset  # Use for training/optimization
val_set = dspy_dataset    # Use for validation/scoring

print(f"\n✓ Single-worksheet optimization mode enabled")
print(f"  Training set: {len(train_set)} item(s)")
print(f"  Validation set: {len(val_set)} item(s)")
print(f"\nGEPA will optimize the prompt specifically for worksheet type {WORKSHEET_INDEX}")

## Configure Language Models

Setup task LLM (Gemini 2.0 Flash Lite) and reflection LLM (GPT-5 for high reasoning).

In [ ]:
# Configure DSPy with Gemini 2.0 Flash Lite for task execution
task_lm = dspy.LM(
    model="gemini/gemini-2.5-flash-lite",  # Task LM - Gemini 2.0 Flash Lite
    max_tokens=16000
)

# Configure reflection LM with GPT-5 (high reasoning capability)
# Note: GPT-5 reasoning models require temperature=1.0 and max_tokens >= 16000
reflection_lm = dspy.LM(
    model="openai/gpt-5",  # Reflection LM - GPT-5 for high reasoning
    temperature=1.0,  # Required for reasoning models
    max_tokens=16000,  # Minimum required for reasoning models
)

print("Language models configured:")
print(f"- Task LM: {task_lm.model}")
print(f"- Reflection LM: {reflection_lm.model} (reasoning mode: temp=1.0, max_tokens=16000)")

## Setup DSPy Adapter for GEPA

Configure the DspyAdapter with the task LM, metric function, and reflection LM.

In [ ]:
# Setup DSPy Adapter
adapter = DspyAdapter(
    task_lm=task_lm,
    metric_fn=metric_fn,
    num_threads=4,  # Adjust based on your system
    reflection_lm=lambda x: reflection_lm(x)[0]
)

print("DSPy Adapter configured successfully!")

## Test Baseline Performance

Test the initial seed program before optimization.

In [ ]:
print("Evaluating baseline performance...")
baseline_result = adapter.evaluate(val_set, {"program": seed_program})
print(f"\nBaseline performance: {baseline_result}")

# Access as object attributes, not dict
if hasattr(baseline_result, 'scores') and baseline_result.scores:
    avg_score = sum(baseline_result.scores) / len(baseline_result.scores)
    print(f"Average score: {avg_score:.4f}")
else:
    print(f"Average score: N/A")

## Run GEPA Optimization

Now run the GEPA optimization process to improve the prompt!

In [ ]:
print(f"Starting GEPA optimization for Worksheet {WORKSHEET_INDEX}...")
print("This will run multiple reasoning iterations to optimize specifically for this worksheet type...")
print(f"Max iterations: 100")

# Run GEPA optimization
optimization_result = optimize(
    seed_candidate={"program": seed_program},
    trainset=train_set,
    valset=val_set,
    adapter=adapter,
    reflection_lm=lambda x: reflection_lm(x)[0],
    max_metric_calls=3,  # Adjust for more/fewer optimization iterations
    display_progress_bar=True
)

print("\n" + "="*80)
print(f"OPTIMIZATION COMPLETE FOR WORKSHEET {WORKSHEET_INDEX}!")
print("="*80)

## View Optimized Program

Let's see what GEPA came up with!

In [ ]:
print("\n" + "="*80)
print("OPTIMIZED PROGRAM")
print("="*80)
print(optimization_result.best_candidate["program"])
print("\n" + "="*80)

## Evaluate Optimized Performance

Compare the optimized program against the validation set.

In [ ]:
print("Evaluating optimized program...")
optimized_result = adapter.evaluate(val_set, optimization_result.best_candidate)

print("\n" + "="*80)
print("PERFORMANCE COMPARISON")
print("="*80)

# Calculate scores from object attributes
baseline_score = sum(baseline_result.scores) / len(baseline_result.scores) if baseline_result.scores else 0
optimized_score = sum(optimized_result.scores) / len(optimized_result.scores) if optimized_result.scores else 0

print(f"Baseline Average Score:  {baseline_score:.4f}")
print(f"Optimized Average Score: {optimized_score:.4f}")

if baseline_score > 0:
    improvement = optimized_score - baseline_score
    improvement_pct = (improvement / baseline_score) * 100
    print(f"\nImprovement: {improvement:.4f} ({improvement_pct:.2f}%)")
print("="*80)

## Test Function with Images

Create a test function that executes a single prompt with images using the optimized program.

In [ ]:
def test_with_images(example_idx: int, use_optimized: bool = True):
    """
    Test a single example from the dataset with images.
    
    Args:
        example_idx: Index of example to test (0-based from validation set)
        use_optimized: If True, use optimized program; otherwise use baseline
    """
    if example_idx >= len(val_set):
        print(f"Error: Example index {example_idx} out of range (max: {len(val_set)-1})")
        return
    
    example = val_set[example_idx]
    
    print("\n" + "="*80)
    print(f"TESTING EXAMPLE {example_idx}")
    print("="*80)
    print(f"\nPrompt: {example.prompt}")
    print(f"Number of images: {len(example.images)}")
    
    # Display images
    for i, img_data in enumerate(example.images, 1):
        print(f"  Image {i}: {img_data['path']}")
        img = Image.open(img_data['path'])
        display(img.resize((min(img.width, 400), min(img.height, 400))))
    
    print("\nExpected Output:")
    print(json.dumps(example.expected_output, indent=2))
    
    # Execute with selected program
    program_code = optimization_result.best_candidate["program"] if use_optimized else seed_program
    program_type = "OPTIMIZED" if use_optimized else "BASELINE"
    
    print(f"\nExecuting with {program_type} program...")
    
    # For simplicity, execute directly with vision API
    # In production, you'd execute the DSPy program
    result = execute_vision_prompt(
        prompt=example.prompt,
        images=example.images,
        system_prompt="Analyze the images and return a valid JSON response matching the prompt requirements."
    )
    
    print(f"\nModel Output ({program_type}):")
    print(result)
    
    # Calculate score
    score = compare_json_outputs(result, example.expected_output)
    print(f"\nMatch Score: {score:.2%}")
    print("="*80)
    
    return result, score


# Example usage - test first validation example
# test_with_images(0, use_optimized=True)

## Save Optimized Program

Save the optimized program for future use.

In [ ]:
# Save the optimized program for this specific worksheet
output_file = f"optimized_worksheet_{WORKSHEET_INDEX}.py"
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(optimization_result.best_candidate["program"])

print(f"💾 Optimized program saved to: {output_file}")

# Also save optimization results as JSON
results_file = f"results_worksheet_{WORKSHEET_INDEX}.json"

# Calculate scores from object attributes
baseline_score = sum(baseline_result.scores) / len(baseline_result.scores) if baseline_result.scores else 0
optimized_score = sum(optimized_result.scores) / len(optimized_result.scores) if optimized_result.scores else 0
improvement = optimized_score - baseline_score
improvement_pct = (improvement / baseline_score * 100) if baseline_score > 0 else 0

results_data = {
    "worksheet_index": WORKSHEET_INDEX,
    "baseline_score": baseline_score,
    "optimized_score": optimized_score,
    "improvement": improvement,
    "improvement_percentage": improvement_pct,
    "max_metric_calls": 100,
    "task_lm": "gemini/gemini-2.0-flash-lite",
    "reflection_lm": "openai/gpt-5",
    "optimized_program": optimization_result.best_candidate["program"]
}

with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(results_data, f, indent=2)

print(f"💾 Optimization results saved to: {results_file}")
print(f"\n✅ Worksheet {WORKSHEET_INDEX} optimization complete!")
print(f"📝 To optimize another worksheet, change WORKSHEET_INDEX in cell 16 and run again.")

## Batch Optimization Helper (Optional)

Use this cell to optimize multiple worksheet types in sequence.

In [ ]:
def optimize_single_worksheet(worksheet_idx: int, max_iterations: int = 100, save_results: bool = True):
    """
    Optimize a single worksheet type individually.
    
    Args:
        worksheet_idx: Index of the worksheet to optimize (e.g., 1, 2, 3)
        max_iterations: Number of GEPA optimization iterations
        save_results: Whether to save optimized program and results
        
    Returns:
        Dictionary with optimization results
    """
    print(f"\n{'='*80}")
    print(f"OPTIMIZING WORKSHEET TYPE {worksheet_idx}")
    print(f"{'='*80}\n")
    
    # Load single worksheet
    try:
        item = load_dataset_item(worksheet_idx, "Dataset")
    except Exception as e:
        print(f"❌ Error loading worksheet {worksheet_idx}: {e}")
        return None
    
    # Prepare dataset
    dataset = prepare_dspy_dataset([item])
    train_set = dataset
    val_set = dataset
    
    print(f"✓ Loaded worksheet {worksheet_idx}")
    print(f"  Images: {len(item['images'])}")
    print(f"  Prompt: {item['prompt'][:100]}...")
    
    # Baseline evaluation
    print(f"\n📊 Evaluating baseline...")
    baseline = adapter.evaluate(val_set, {"program": seed_program})
    baseline_score = sum(baseline.scores) / len(baseline.scores) if baseline.scores else 0
    print(f"  Baseline score: {baseline_score:.4f}")
    
    # Run optimization
    print(f"\n🚀 Starting GEPA optimization ({max_iterations} iterations)...")
    result = optimize(
        seed_candidate={"program": seed_program},
        trainset=train_set,
        valset=val_set,
        adapter=adapter,
        reflection_lm=lambda x: reflection_lm(x)[0],
        max_metric_calls=max_iterations,
        display_progress_bar=True
    )
    
    # Evaluate optimized
    print(f"\n📊 Evaluating optimized program...")
    optimized = adapter.evaluate(val_set, result.best_candidate)
    optimized_score = sum(optimized.scores) / len(optimized.scores) if optimized.scores else 0
    
    # Results
    improvement = optimized_score - baseline_score
    improvement_pct = (improvement / baseline_score * 100) if baseline_score > 0 else 0
    
    print(f"\n{'='*80}")
    print(f"RESULTS FOR WORKSHEET {worksheet_idx}")
    print(f"{'='*80}")
    print(f"Baseline:   {baseline_score:.4f}")
    print(f"Optimized:  {optimized_score:.4f}")
    print(f"Improvement: {improvement:+.4f} ({improvement_pct:+.2f}%)")
    print(f"{'='*80}\n")
    
    # Save if requested
    if save_results:
        # Save optimized program
        program_file = f"optimized_worksheet_{worksheet_idx}.py"
        with open(program_file, 'w', encoding='utf-8') as f:
            f.write(result.best_candidate["program"])
        print(f"💾 Saved program: {program_file}")
        
        # Save results
        results_file = f"results_worksheet_{worksheet_idx}.json"
        results_data = {
            "worksheet_index": worksheet_idx,
            "baseline_score": baseline_score,
            "optimized_score": optimized_score,
            "improvement": improvement,
            "improvement_percentage": improvement_pct,
            "max_iterations": max_iterations,
            "task_lm": "gemini/gemini-2.0-flash-lite",
            "reflection_lm": "openai/gpt-5",
            "optimized_program": result.best_candidate["program"]
        }
        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=2)
        print(f"💾 Saved results: {results_file}")
    
    return {
        "worksheet_idx": worksheet_idx,
        "baseline": baseline_score,
        "optimized": optimized_score,
        "improvement": improvement,
        "program": result.best_candidate["program"]
    }


# ============================================================
# BATCH OPTIMIZATION: Optimize multiple worksheets
# ============================================================
# Uncomment and run this to optimize multiple worksheets sequentially

# worksheet_indices = [1, 2, 3]  # Add all worksheet indices you want to optimize
# results_summary = []
# 
# for idx in worksheet_indices:
#     result = optimize_single_worksheet(
#         worksheet_idx=idx,
#         max_iterations=100,  # Adjust as needed
#         save_results=True
#     )
#     if result:
#         results_summary.append(result)
#     print("\n" + "="*80 + "\n")
# 
# # Print summary
# print("\n" + "="*80)
# print("OPTIMIZATION SUMMARY FOR ALL WORKSHEETS")
# print("="*80)
# for r in results_summary:
#     print(f"Worksheet {r['worksheet_idx']}: {r['baseline']:.4f} → {r['optimized']:.4f} ({r['improvement']:+.4f})")
# print("="*80)

## Compare Results Across Worksheets

View optimization results for all worksheets you've processed.

In [ ]:
def compare_all_worksheets():
    """
    Load and compare results from all optimized worksheets.
    Displays a summary table of improvements.
    """
    import glob
    
    print("\n" + "="*100)
    print(" "*35 + "WORKSHEET OPTIMIZATION SUMMARY")
    print("="*100)
    print(f"{'Worksheet':<12} {'Baseline':<12} {'Optimized':<12} {'Improvement':<15} {'% Change':<12} {'Status':<10}")
    print("-"*100)
    
    # Find all result files
    result_files = sorted(glob.glob("results_worksheet_*.json"))
    
    if not result_files:
        print("No optimization results found. Run optimizations first.")
        return
    
    total_improvements = []
    
    for result_file in result_files:
        try:
            with open(result_file, 'r') as f:
                data = json.load(f)
            
            ws_idx = data.get('worksheet_index', 'N/A')
            baseline = data.get('baseline_score', 0)
            optimized = data.get('optimized_score', 0)
            improvement = data.get('improvement', 0)
            pct_change = data.get('improvement_percentage', 0)
            
            # Status indicator
            if improvement > 0.1:
                status = "🟢 Great"
            elif improvement > 0:
                status = "🟡 Good"
            else:
                status = "🔴 Review"
            
            total_improvements.append(improvement)
            
            print(f"{'WS-' + str(ws_idx):<12} {baseline:<12.4f} {optimized:<12.4f} {improvement:+<15.4f} {pct_change:+<12.2f}% {status:<10}")
            
        except Exception as e:
            print(f"Error reading {result_file}: {e}")
    
    print("-"*100)
    if total_improvements:
        avg_improvement = sum(total_improvements) / len(total_improvements)
        print(f"Average Improvement: {avg_improvement:+.4f} across {len(total_improvements)} worksheet(s)")
    print("="*100 + "\n")


# Run this to see comparison of all optimized worksheets
# compare_all_worksheets()

## Summary and Usage Guide

### 🎯 Single-Worksheet Optimization Mode

This notebook is configured for **individual worksheet optimization** - optimizing prompts for one specific worksheet type at a time, without generalization.

### What This Does:

1. **Loads Single Worksheet**: Reads one specific worksheet (e.g., worksheet 1, 2, or 3)
2. **Focused Optimization**: GEPA optimizes specifically for that worksheet type
3. **Multiple Iterations**: Uses reasoning model (GPT-5) to iteratively improve the prompt
4. **Saves Results**: Each worksheet gets its own optimized program file
5. **No Generalization**: Each worksheet type is treated independently

### 📋 Workflow:

#### **Option 1: Optimize One Worksheet at a Time (Manual)**

1. **Set the worksheet index** in cell 16:
   ```python
   WORKSHEET_INDEX = 1  # Change to 2, 3, etc. for other worksheets
   ```

2. **Run all cells** to optimize that specific worksheet

3. **Results saved as**:
   - `optimized_program.py` - The optimized DSPy program
   - `optimization_results.json` - Performance metrics

4. **Repeat for other worksheets**:
   - Change `WORKSHEET_INDEX = 2`
   - Run all cells again
   - Continue for worksheet 3, 4, etc.

#### **Option 2: Batch Optimize Multiple Worksheets (Automated)**

Use the batch helper function in the last cell:

```python
# Optimize worksheets 1, 2, and 3 sequentially
worksheet_indices = [1, 2, 3]
results_summary = []

for idx in worksheet_indices:
    result = optimize_single_worksheet(
        worksheet_idx=idx,
        max_iterations=100,  # More iterations = better optimization
        save_results=True
    )
    if result:
        results_summary.append(result)

# View summary of all optimizations
for r in results_summary:
    print(f"Worksheet {r['worksheet_idx']}: {r['baseline']:.4f} → {r['optimized']:.4f}")
```

**Saves for each worksheet**:
- `optimized_worksheet_1.py`
- `results_worksheet_1.json`
- `optimized_worksheet_2.py`
- `results_worksheet_2.json`
- etc.

### 🔧 Key Configuration:

**Language Models** (cell 18):
- Task LLM: `Gemini 2.0 Flash Lite` (processes images and executes tasks)
- Reflection LLM: `GPT-5` (reasons about improvements)

**Optimization Iterations** (cell 24 or in batch function):
- `max_metric_calls=100` - More iterations = better optimization
- Adjust based on time/budget

### 📁 Dataset Structure:

```
Dataset/
├── prompts/
│   ├── 1.txt  (worksheet type 1)
│   ├── 2.txt  (worksheet type 2)
│   ├── 3.txt  (worksheet type 3)
│   └── ...
├── outputs/
│   ├── 1.json (expected output for worksheet 1)
│   ├── 2.json
│   └── ...
└── images/
    ├── 1-a.png, 1-b.jpg (images for worksheet 1 - PNG or JPG)
    ├── 2-a.jpg, 2-b.jpeg (images for worksheet 2)
    └── ...
```

**Image formats supported**: PNG, JPG, JPEG (case-insensitive)

### 💡 Why This Approach?

- ✅ **Specialized optimization** for each worksheet type
- ✅ **No generalization needed** - each prompt is unique
- ✅ **Deep optimization** - multiple reasoning iterations on single type
- ✅ **Easy iteration** - optimize worksheet 1, then 2, then 3, etc.
- ✅ **Individual tracking** - separate results for each worksheet

### 🚀 Quick Start:

1. Set `WORKSHEET_INDEX = 1` in cell 16
2. Click "Run All"
3. Wait for optimization to complete
4. Check `optimized_worksheet_1.py` for results
5. Repeat for other worksheets!